# Gestational Diabetes Meal Risk Predictor
## Notebook 1: Data Cleaning and Exploratory Data Analysis

**Author:** Sanja  
**Date:** December 25, 2025  
**Project:** Springboard Capstone - Predicting Post-Meal Glucose Risk

---

### Objectives:
1. Load nutritional data from USDA FoodData Central
2. Load glycemic index reference tables
3. Merge and clean the datasets
4. Perform exploratory data analysis
5. Understand feature distributions and relationships
6. Prepare data for feature engineering

### Data Sources:
- **USDA FoodData Central**: Nutritional composition of foods
- **Glycemic Index Tables**: GI/GL values from published research

## 1. Import Required Libraries

Import all necessary libraries for data processing, analysis, and visualization.

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Utilities
import os
import warnings
from pathlib import Path

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("✓ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Set Up Data Paths

Define paths to data directories and check if data files exist.

In [ ]:
# Define project root and data directories
PROJECT_ROOT = Path(os.getcwd()).parent
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
REPORTS_FIGURES = PROJECT_ROOT / 'reports' / 'figures'

# Create directories if they don't exist
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
REPORTS_FIGURES.mkdir(parents=True, exist_ok=True)

print(f"Project Root: {PROJECT_ROOT}")
print(f"Raw Data Path: {DATA_RAW}")
print(f"Processed Data Path: {DATA_PROCESSED}")
print(f"\nDirectory structure created/verified ✓")

## 3. Data Collection Instructions

### Next Steps: Download Data Files

Before proceeding with the analysis, you need to download the following datasets:

#### A. USDA FoodData Central
1. Visit: https://fdc.nal.usda.gov/download-datasets.html
2. Download: **"FoodData Central CSV"** (Foundation Foods or SR Legacy)
3. Extract and save `food.csv` and `food_nutrient.csv` to `data/raw/`
4. Alternative: Use the FoodData Central API for programmatic access

#### B. Glycemic Index Reference Table
1. Option 1: Download from University of Sydney GI Database
   - Visit: https://glycemicindex.com/
2. Option 2: Use compiled GI tables from published research
   - Example: Atkinson et al. (2008) International Tables of GI and GL
3. Save as `gi_table.csv` in `data/raw/`

Expected columns in GI table:
- `food_name`: Name of the food item
- `glycemic_index`: GI value (0-100+)
- `glycemic_load`: GL value (optional, can be calculated)
- `category`: Food category (e.g., grains, fruits, dairy)

#### File Structure Expected:
```
data/raw/
├── food.csv              # USDA food items
├── food_nutrient.csv     # USDA nutritional values
└── gi_table.csv          # Glycemic index reference
```

**Note:** Once you've downloaded these files, uncomment and run the data loading cells below.

## 4. Load USDA Nutritional Data

Load and explore the USDA FoodData Central dataset containing nutritional information.

In [ ]:
# Load USDA food data (uncomment after downloading)
# usda_foods = pd.read_csv(DATA_RAW / 'food.csv', low_memory=False)
# usda_nutrients = pd.read_csv(DATA_RAW / 'food_nutrient.csv', low_memory=False)

# For now, create a sample dataset to demonstrate the workflow
# TODO: Replace with actual USDA data once downloaded

# Sample data structure
sample_foods = pd.DataFrame({
    'fdc_id': [1001, 1002, 1003, 1004, 1005],
    'description': ['White Rice, cooked', 'Brown Rice, cooked', 'Whole Wheat Bread', 
                    'Apple, raw', 'Chicken Breast, grilled'],
    'data_type': ['foundation_food'] * 5
})

print("Sample Food Data Structure:")
print(sample_foods.head())
print(f"\nShape: {sample_foods.shape}")
print("\n⚠️ Remember to load actual USDA data!")

## 5. Load Glycemic Index Reference Data

Load the glycemic index table with GI/GL values for various foods.

In [ ]:
# Load GI table (uncomment after downloading)
# gi_data = pd.read_csv(DATA_RAW / 'gi_table.csv')

# Sample GI data structure
sample_gi_data = pd.DataFrame({
    'food_name': ['White Rice', 'Brown Rice', 'Whole Wheat Bread', 'Apple', 'Chicken Breast'],
    'glycemic_index': [73, 50, 74, 36, 0],
    'gi_category': ['high', 'medium', 'high', 'low', 'low'],
    'category': ['grains', 'grains', 'grains', 'fruits', 'protein']
})

print("Sample Glycemic Index Data:")
print(sample_gi_data)
print(f"\nGI Categories Distribution:")
print(sample_gi_data['gi_category'].value_counts())
print("\n⚠️ Remember to load actual GI reference table!")

## 6. Data Inspection and Quality Check

Examine data structure, missing values, and data types.

In [ ]:
def inspect_dataframe(df, name):
    """Comprehensive inspection of a dataframe"""
    print(f"\n{'='*60}")
    print(f"Dataset: {name}")
    print(f"{'='*60}")
    print(f"\nShape: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"\nData Types:\n{df.dtypes}")
    print(f"\nMissing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Percentage': missing_pct
    }).sort_values('Missing Count', ascending=False)
    print(missing_df[missing_df['Missing Count'] > 0])
    print(f"\nFirst Few Rows:")
    print(df.head(3))
    print(f"\nBasic Statistics:")
    print(df.describe())
    
# Inspect sample data
inspect_dataframe(sample_foods, "USDA Foods (Sample)")
inspect_dataframe(sample_gi_data, "Glycemic Index (Sample)")

## 7. Create Comprehensive Nutritional Dataset

Build a complete nutritional profile for each food item including all key features needed for modeling.

In [ ]:
# Create a sample comprehensive dataset with all nutritional features
# This demonstrates the structure you'll create from actual USDA data

np.random.seed(42)
n_samples = 200

# Sample food dataset with realistic nutritional values
foods_df = pd.DataFrame({
    'food_name': [
        'White Rice', 'Brown Rice', 'Quinoa', 'White Bread', 'Whole Wheat Bread',
        'Sweet Potato', 'Potato', 'Banana', 'Apple', 'Orange',
        'Chicken Breast', 'Salmon', 'Tofu', 'Lentils', 'Black Beans',
        'Broccoli', 'Spinach', 'Carrots', 'Soda', 'Orange Juice'
    ] * 10
})

# Add nutritional features (per 100g serving)
foods_df['total_carbs_g'] = np.random.uniform(0, 80, n_samples)
foods_df['fiber_g'] = np.random.uniform(0, 15, n_samples)
foods_df['sugar_g'] = np.random.uniform(0, 30, n_samples)
foods_df['protein_g'] = np.random.uniform(0, 30, n_samples)
foods_df['fat_g'] = np.random.uniform(0, 20, n_samples)
foods_df['saturated_fat_g'] = foods_df['fat_g'] * 0.3
foods_df['energy_kcal'] = np.random.uniform(50, 400, n_samples)
foods_df['glycemic_index'] = np.random.uniform(15, 90, n_samples)

print("Sample Comprehensive Nutritional Dataset:")
print(foods_df.head(10))
print(f"\nDataset Shape: {foods_df.shape}")
print(f"\nFeature Summary:\n{foods_df.describe()}")

## 8. Exploratory Data Analysis: Feature Distributions

Visualize the distributions of key nutritional features.

In [ ]:
# Plot distributions of key nutritional features
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
fig.suptitle('Distribution of Nutritional Features', fontsize=16, fontweight='bold')

features = ['total_carbs_g', 'fiber_g', 'sugar_g', 'protein_g', 
            'fat_g', 'saturated_fat_g', 'energy_kcal', 'glycemic_index']

for idx, feature in enumerate(features):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    ax.hist(foods_df[feature], bins=30, edgecolor='black', alpha=0.7)
    ax.set_title(f'{feature.replace("_", " ").title()}', fontweight='bold')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.grid(True, alpha=0.3)

# Remove empty subplot
axes[2, 2].axis('off')

plt.tight_layout()
plt.show()

print("✓ Distribution plots created")

## 9. Correlation Analysis

Examine correlations between nutritional features to identify multicollinearity and relationships.

In [ ]:
# Select numerical features for correlation analysis
numeric_features = ['total_carbs_g', 'fiber_g', 'sugar_g', 'protein_g', 
                    'fat_g', 'saturated_fat_g', 'energy_kcal', 'glycemic_index']

# Calculate correlation matrix
correlation_matrix = foods_df[numeric_features].corr()

# Create heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, 
            annot=True, 
            fmt='.2f', 
            cmap='coolwarm', 
            center=0,
            square=True,
            linewidths=1,
            cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: Nutritional Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Identify high correlations
print("\nHighly Correlated Feature Pairs (|r| > 0.7):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            print(f"  {correlation_matrix.columns[i]} <-> {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")

## 10. Glycemic Index Analysis

Analyze the distribution of glycemic index values and their relationship with other features.

In [ ]:
# Categorize GI values
def categorize_gi(gi_value):
    if gi_value < 55:
        return 'Low'
    elif gi_value < 70:
        return 'Medium'
    else:
        return 'High'

foods_df['gi_category'] = foods_df['glycemic_index'].apply(categorize_gi)

# Plot GI distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# GI Distribution
axes[0].hist(foods_df['glycemic_index'], bins=20, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].axvline(x=55, color='orange', linestyle='--', linewidth=2, label='Low/Medium threshold')
axes[0].axvline(x=70, color='red', linestyle='--', linewidth=2, label='Medium/High threshold')
axes[0].set_title('Glycemic Index Distribution', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Glycemic Index')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# GI Categories
gi_counts = foods_df['gi_category'].value_counts()
axes[1].bar(gi_counts.index, gi_counts.values, edgecolor='black', alpha=0.7)
axes[1].set_title('GI Category Distribution', fontweight='bold', fontsize=12)
axes[1].set_xlabel('GI Category')
axes[1].set_ylabel('Count')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nGI Category Distribution:")
print(foods_df['gi_category'].value_counts())
print(f"\nGI Statistics:")
print(foods_df['glycemic_index'].describe())

## 11. Key Insights and Data Quality Summary

Summarize findings from the exploratory analysis and note any data quality issues.

In [ ]:
print("=" * 80)
print("KEY FINDINGS FROM EXPLORATORY DATA ANALYSIS")
print("=" * 80)

print("\n📊 Data Quality:")
print(f"   • Total food items: {len(foods_df)}")
print(f"   • Features available: {len(foods_df.columns)}")
print(f"   • Missing values: {foods_df.isnull().sum().sum()}")
print(f"   • Duplicate rows: {foods_df.duplicated().sum()}")

print("\n🥗 Nutritional Feature Ranges:")
for feature in ['total_carbs_g', 'fiber_g', 'protein_g', 'fat_g']:
    print(f"   • {feature}: {foods_df[feature].min():.1f} - {foods_df[feature].max():.1f}g")

print(f"\n📈 Glycemic Index:")
print(f"   • Range: {foods_df['glycemic_index'].min():.1f} - {foods_df['glycemic_index'].max():.1f}")
print(f"   • Mean: {foods_df['glycemic_index'].mean():.1f}")
print(f"   • Low GI foods: {(foods_df['gi_category'] == 'Low').sum()}")
print(f"   • Medium GI foods: {(foods_df['gi_category'] == 'Medium').sum()}")
print(f"   • High GI foods: {(foods_df['gi_category'] == 'High').sum()}")

print("\n⚠️  Data Quality Issues to Address:")
print("   1. Check for and handle missing GI values")
print("   2. Standardize food name variations for merging")
print("   3. Validate nutritional value ranges (outliers)")
print("   4. Ensure serving sizes are consistent")
print("   5. Handle zero/null values in key features")

print("\n✅ Next Steps:")
print("   1. Download actual USDA FoodData Central dataset")
print("   2. Download comprehensive GI reference table")
print("   3. Proceed to Notebook 02: Feature Engineering")
print("   4. Create synthetic risk labels based on GI + macros")

print("\n" + "=" * 80)

## 12. Save Cleaned Data (Placeholder)

Once you have real data loaded and cleaned, save it for the next notebook.

In [ ]:
# Save the cleaned dataset for feature engineering
# Uncomment when you have real data

# output_file = DATA_PROCESSED / 'foods_cleaned.csv'
# foods_df.to_csv(output_file, index=False)
# print(f"✓ Cleaned data saved to: {output_file}")

# For now, save sample data
sample_output = DATA_PROCESSED / 'sample_foods_cleaned.csv'
foods_df.to_csv(sample_output, index=False)
print(f"✓ Sample data saved to: {sample_output}")
print(f"\n📝 Ready for Notebook 02: Feature Engineering!")